In [47]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import QuantileRegressor
from sklearn.linear_model import LinearRegression 

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [11]:
base2024 = pd.read_excel('EPH_usu_1er_Trim_2024_xlsx/usu_individual_T124.xlsx',engine='openpyxl')

In [36]:
base2024 = base2024.query(
    "P21 > 0 and CH03 == 1 and CH04 == 1 and 25 <= CH06 <= 65 and CH12 !=0 and CH12 != 9"
)
previo = np.array([0, 0, 6, 6, 9, 12, 12, 18])
posterior = np.array([0, 6, 9, 12, 12, 15, 18, 22])

def faux(row): 
    if row['CH03'] == 1: 
        return posterior[row['CH12']-1] 
    else: 
        return previo[row['CH12']-1] + int(row['CH14']) 

base2024['anios_ed'] = base2024.apply(faux, axis=1)

base2024['pot_exp'] = base2024['CH06'] - base2024['anios_ed'] - 6
base2024['pot_exp_2'] = base2024['pot_exp'] ** 2

In [53]:
#rq <- rq(log(P21) ~ anios_ed + pot_exp + pot_exp_2 + as.factor(AGLOMERADO) + as.factor(CH07),data=base2,tau = .5,ci = FALSE)

prepocessor = ColumnTransformer(
    transformers = [
        ('cat',OneHotEncoder(),['AGLOMERADO','CH07'])
    ],
    remainder='passthrough'
)
model_linear_reg = Pipeline(steps=[
    ('preprocessor',prepocessor),
    ('regressor',LinearRegression())
])
QuantReg = QuantileRegressor()

X = base2024[['anios_ed', 'pot_exp', 'pot_exp_2', 'AGLOMERADO', 'CH07']]
y = np.log(base2024['P21'])
model_linear_reg.fit(X,y)

/home/kenetc/.cache/pypoetry/virtualenvs/econometria-_nRykAoR-py3.10/lib/python3.10/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['AGLOMERADO', 'CH07'])])),
                ('regressor', LinearRegression())])

3        12.429216
10       12.206073
20       12.765688
29       12.468437
44       12.899220
           ...    
46014    11.512925
46022    12.100712
46026    13.017003
46028    13.710150
46031    12.765688
Name: P21, Length: 4530, dtype: float64